# Accuracy of the models

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
import pandas as pd
from vessel_manoeuvring_models.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)
from vessel_manoeuvring_models.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)S

In [ ]:
regressions = global_variables['regressions']

regression_names = ['motion_regression','force_regression']

def load_joined(join='joined'):
    df_accuracies = pd.DataFrame()
    
    for vmm_name in vmms:
        for model_test_id in model_test_ids:
            
            file_paths = []
            file_paths.append(f"{ vmm_name }.motion_regression.{join}.{model_test_id}.simulation_accuracy")
            file_paths.append(f"{ vmm_name }.force_regression.{model_test_id}.simulation_accuracy")
            
            
            for regression,file_path in zip(regression_names, file_paths):
                try:
                    accuracy = catalog.load(file_path)
                except:
                    continue
                
                accuracy = pd.Series(accuracy)
                accuracy['total'] = accuracy.mean()
                
                accuracy['vmm'] = vmm_name
                accuracy['regression'] = regression
                accuracy['model test'] = model_test_id
                
                df_accuracies = df_accuracies.append(accuracy, ignore_index=True)
                
    return df_accuracies
            

In [ ]:
df_accuracies = load_joined()

In [ ]:
df_compare = df_accuracies.groupby(by=['vmm','regression']).mean().sort_values(by='total', ascending=True)
df_compare

In [ ]:
df_compare['total'].plot.bar()

In [ ]:
df_compare.reset_index().pivot(index=['vmm'], columns=['regression'], values='total').plot.bar();

In [ ]:
df_compare.groupby('regression').get_group('motion_regression').plot.bar(y=['x0','y0','psi'])

## Online prediction accuracy

In [ ]:
regression = "motion_regression"

df_rmse = pd.DataFrame() 

for vmm in vmms:
    
    for join in joins:
    
        for id in model_test_ids:

            rmse = catalog.load(f"{ vmm }.{ regression }.{ join }.{ id }.online_prediction_rmse")
            
            
            rmse["vmm"] = vmm
            rmse["join"] = join
            rmse["model_test_id"] = id
            df_rmse = df_rmse.append(rmse, ignore_index=True)
            
            
            

In [ ]:
df_rmse

In [ ]:
df_rmse.groupby(by=['vmm','join']).mean().plot.bar()

In [ ]:
df_rmse.groupby(by=['join']).get_group('joined').groupby(by=['vmm']).mean().plot.bar()

In [ ]:
df_compare = df_rmse.groupby(by=['join']).get_group('joined').copy()
#df_compare.set_index('model_test_id', inplace=True)
df_compare.pivot(index='model_test_id', columns='vmm').plot.bar(y='A_FP')

In [ ]:
id = 22774

df = catalog.load(f"{ id }.data_ek_smooth")
#df = catalog.load(f"{ id }.data")

df_online_errors = pd.DataFrame()

keys=['u1d','v1d','r1d']
#keys=['x0','y0','psi']

data = df[keys].copy()

for vmm in vmms:
    df_ = catalog.load(f"{ vmm }.{ regression }.{ join }.{ id }.data_online_predict")
    df_ = df_[keys].copy()
    df_= (df_ - data.values)/data.abs().max()
    df_['vmm'] = vmm
    df_['model_test_id'] = id
    df_online_errors = df_online_errors.append(df_)
    


In [ ]:
df_online_errors.head()

In [ ]:
dataframes={key:value for key,value in df_online_errors.groupby(['vmm','model_test_id'])}
styles={key:{'alpha':0.9} for key,value in df_online_errors.groupby(['vmm','model_test_id'])}


plot(dataframes, keys=keys,styles=styles);